# <a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science</a>
# <a href=https://ramp.r0h.eu/problems/air_passengers>RAMP</a> on predicting the number of air passengers

<i> Balázs Kégl (LAL/CNRS), Alex Gramfort (LTCI/Telecom ParisTech), Djalel Benbouzid (UPMC), Mehdi Cherti (LAL/CNRS) </i>

## Introduction
The data set was donated to us by an unnamed company handling flight ticket reservations. The data is thin, it contains
<ul>
<li> the date of departure
<li> the departure airport
<li> the arrival airport
<li> the mean and standard deviation of the number of weeks of the reservations made before the departure date
<li> a field called <code>log_PAX</code> which is related to the number of passengers (the actual number were changed for privacy reasons)
</ul>

The goal is to predict the <code>log_PAX</code> column. The prediction quality is measured by RMSE. 

The data is obviously limited, but since data and location informations are available, it can be joined to external data sets. <b>The challenge in this RAMP is to find good data that can be correlated to flight traffic</b>.

To do :
- mean encoding 
- regrouper variables catégoriques
- ajouter nouvelles données : distance aéroports (trouver fichier avec données)
- faire analyses descriptives

In [1]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

## Fetch the data and load it in pandas

First we load `problem.py` that parameterizes the challenge. It contains some objects taken off the shelf from `ramp-workflow` (e.g., `Predictions` type, scores, and data reader). 

In [2]:
problem = imp.load_source('', 'problem.py')

`get_train_data` loads the training data and returns an `pandas` object (input) and a `np.array` object (output).

In [3]:
X_df, y_array = problem.get_train_data()
# the y array contains the log-like number of passengers on each flight 

In [4]:
print(min(X_df['DateOfDeparture']))
print(max(X_df['DateOfDeparture']))

2011-09-01
2013-03-05


In [5]:
X_df['log_PAX'] = y_array
cols = X_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
X_df = X_df[cols]
X_df.head()

,log_PAX,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,12.331296,2012-06-19,ORD,DFW,12.875000,9.812647
1,10.775182,2012-09-10,LAS,DEN,14.285714,9.466734
2,11.083177,2012-10-05,DEN,LAX,10.863636,9.035883
3,11.169268,2011-10-09,ATL,ORD,11.480000,7.990202
4,11.269364,2012-02-21,DEN,SFO,11.450000,9.517159


In [6]:
data = pd.read_csv('submissions/starting_kit/external_data.csv')

In [7]:
X_encoded = X_df
external_data = data[['DateOfDeparture', 'Departure','Arrival', 'Mean TemperatureC', 'Distance',
                      'MeanDew PointC', 'Mean Humidity', 'Min VisibilitykM', 'Mean Sea Level PressurehPa',
                      'Max Wind SpeedKm/h', 
                      'Precipitationmm', 'CloudCover', 'Events','dep_encod','ar_encod','Number_hab','Revenue']]
X_encoded = pd.merge(
            X_encoded, external_data, how='left',
            left_on=['DateOfDeparture', 'Arrival','Departure'],
            right_on=['DateOfDeparture', 'Arrival','Departure'],
            sort=False)
X_encoded.head()

,log_PAX,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Mean TemperatureC,Distance,MeanDew PointC,Mean Humidity,Min VisibilitykM,Mean Sea Level PressurehPa,Max Wind SpeedKm/h,Precipitationmm,CloudCover,Events,dep_encod,ar_encod,Number_hab,Revenue
0,12.331296,2012-06-19,ORD,DFW,12.875000,9.812647,29,1292,21,63,16,1010,48,0.00,5,NaN,11.27,11.08,1241700,45616
1,10.775182,2012-09-10,LAS,DEN,14.285714,9.466734,25,1010,-6,14,16,1008,35,0.00,3,NaN,10.76,10.69,634542,52262
2,11.083177,2012-10-05,DEN,LAX,10.863636,9.035883,19,1387,16,77,8,1016,24,0.00,5,Fog,10.68,11.43,3852782,57739
3,11.169268,2011-10-09,ATL,ORD,11.480000,7.990202,19,977,10,58,16,1026,23,0.00,1,NaN,11.00,11.26,2712920,52273
4,11.269364,2012-02-21,DEN,SFO,11.450000,9.517159,12,1554,8,79,3,1025,24,0.00,7,NaN,10.68,11.23,827420,57739


In [8]:
X_encoded.drop(['std_wtd'], axis=1, inplace=True)

In [9]:
from sklearn.preprocessing import StandardScaler

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)

X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])

X_encoded['year']    = X_encoded['DateOfDeparture'].dt.year
X_encoded['month']   = X_encoded['DateOfDeparture'].dt.month
X_encoded['day']     = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week']    = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days']  = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))

X_encoded.drop(['year', 'month', 'day', 'weekday', 'week', 'n_days','DateOfDeparture'], axis=1, inplace=True)


In [10]:
X_encoded['Events'].fillna(0, inplace=True)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain','Fog'], 1)
X_encoded['Events'] = X_encoded['Events'].replace(['Rain-Thunderstorm','Fog-Rain-Thunderstorm', 
                                                         'Rain-Snow','Snow','Fog-Rain','Thunderstorm','Fog-Snow', 
                                                         'Fog-Rain-Snow','Fog-Rain-Snow-Thunderstorm', 
                                                         'Rain-Snow-Thunderstorm','Rain-Hail-Thunderstorm', 
                                                         'Fog-Rain-Hail-Thunderstorm', 
                                                         'Rain-Thunderstorm-Tornado'], 2)

In [11]:
X_encoded['Precipitationmm'] = X_encoded['Precipitationmm'].replace('T', np.nan)
X_encoded['Precipitationmm'] = pd.to_numeric(X_encoded['Precipitationmm'])
X_encoded['Precipitationmm'] = X_encoded['Precipitationmm'].fillna(X_encoded['Precipitationmm'].mean())

In [12]:
X_encoded['Precipitationmm*CloudCover'] = X_encoded.Precipitationmm * X_encoded.CloudCover
X_encoded['Temperature*Humidity'] = X_encoded['Mean TemperatureC'] * X_encoded['Mean Humidity']

In [13]:
X_encoded['Weeks_to_dep_int'] = pd.qcut(X_encoded['WeeksToDeparture'], 4)
X_encoded.loc[X_encoded['WeeksToDeparture'] <= 9.524, 'WeeksToDeparture'] = 0
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 9.524) & (X_encoded['WeeksToDeparture'] <= 11.3), 'WeeksToDeparture'] = 1
X_encoded.loc[(X_encoded['WeeksToDeparture'] > 11.3) & (X_encoded['WeeksToDeparture'] <= 13.24), 'WeeksToDeparture'] = 2
X_encoded.loc[ X_encoded['WeeksToDeparture'] > 13.24, 'WeeksToDeparture'] = 3
X_encoded['WeeksToDeparture'] = X_encoded['WeeksToDeparture'].astype(int)
X_encoded.drop(['Weeks_to_dep_int'], axis=1, inplace=True)

In [14]:
X_encoded.loc[:, 'Precipitationmm*CloudCover' : 'Temperature*Humidity'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'Precipitationmm*CloudCover' : 'Temperature*Humidity'])

X_encoded.loc[:, 'Distance' : 'Max Wind SpeedKm/h'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'Distance' : 'Max Wind SpeedKm/h'])

X_encoded.loc[:, 'dep_encod' : 'Revenue'] = StandardScaler().fit_transform(
                                                            X_encoded.loc[:, 'dep_encod' : 'Revenue'])

In [15]:
X_encoded.drop(['Mean TemperatureC','Mean Humidity','Precipitationmm','CloudCover'], axis=1, inplace=True)
X_encoded.drop(['MeanDew PointC', 'Mean Sea Level PressurehPa'], axis=1, inplace=True)
X_encoded.drop(['ar_encod'], axis=1, inplace=True)

In [16]:
y = X_encoded['log_PAX']
X_encoded.drop(['log_PAX'], axis=1, inplace=True)

In [17]:
X_encoded.head()

,WeeksToDeparture,Distance,Min VisibilitykM,Max Wind SpeedKm/h,Events,dep_encod,Number_hab,Revenue,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO,y_2011,y_2012,y_2013,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,w_40,w_41,w_42,w_43,w_44,w_45,w_46,w_47,w_48,w_49,w_50,w_51,w_52,Precipitationmm*CloudCover,Temperature*Humidity
0,2,-0.394712,0.784753,1.593843,0,1.003612,-0.297994,-0.964522,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.309833,1.547196
1,3,-0.655678,0.784753,0.372957,0,-0.882932,-0.565232,-0.045009,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.309833,-0.855670
2,1,-0.306797,-0.611095,-0.660101,1,-1.178860,0.851261,0.712766,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-0.309833,0.955021
3,2,-0.686217,0.784753,-0.754015,0,0.004853,0.349556,-0.043487,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-0.309833,0.367726
4,2,-0.152253,-1.483500,-0.660101,0,-1.178860,-0.480338,0.712766,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.309833,0.117190


In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, y, test_size=0.2)

In [19]:
import xgboost as xgb
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4],
              'objective':['reg:squarederror'],
              'learning_rate': [.2], 
              'max_depth': [7],
              'min_child_weight': [4],
              'subsample': [0.95],
              'colsample_bytree': [0.55],
              'n_estimators': [100,300,500,1000],
              'reg_lambda':[0.001],
              'gamma':[0]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, Y_train)

print(xgb_grid.best_params_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 out of   8 | elapsed:  4.1min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   8 out of   8 | elapsed:  4.1min finished
//anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


{'colsample_bytree': 0.55, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 300, 'nthread': 4, 'objective': 'reg:squarederror', 'reg_lambda': 0.001, 'subsample': 0.95}


In [20]:
xgb = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                       colsample_bytree=1, gamma=0, learning_rate=0.3, max_delta_step=0,
                       max_depth=7, min_child_weight=1, missing=None, n_estimators=1000,
                       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
                       reg_alpha=0.001, reg_lambda=0, seed=None,
                       silent=True, subsample=1)
xgb.fit(X_train, Y_train)
Y_pred = xgb.predict(X_test)

KeyboardInterrupt: 

In [ ]:
from collections import OrderedDict
OrderedDict(sorted(xgb.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))